# 2024年数据分析与机器学习课程
## 期末项目任务
### 慈善机构评分预测
#### 一、题目
利用ascore, category，description，……，program_exp，fund_exp，admin_exp等23个特征，预测出score值，这是一个回归问题。
#### 二、训练集
训练集中共有7400个样本，每个样本有23个特征，特征具体介绍如下：
ascore：问责制和透明度得分（满分 100）。
category：机构类别，包含艺术，文化，人文，宗教和教育等等类别。
description：机构介绍，即机构简介。
ein：机构编号，每个机构编号唯一。
tot_exp：总费用（以美元计）（由计划 + 资金 + 行政这三部分组成）。
admin_exp_p：行政费用百分比（占总费用）。
fund_eff：以美元为单位的资金效率（筹集 1 美元捐款所花费的金额）。
fund_exp_p：资金费用百分比（总费用）。
program_exp_p：计划费用百分比（总费用）。
fscore：财务评分（满分 100）。
leader：机构领导者姓名。
leader_comp：机构领导者的薪酬（美元）。
leader_comp_p：领导百分比薪酬（即领导的薪酬占总薪酬的百分比）。
motto：标语，每个机构的宣传标语。
name：慈善机构名称。
tot_rev：该机构的总收入（美元）。
score：该机构所得的总分（满分 100）。
state：该机构在美国的哪个州。
subcategory：子类，即该慈善机构还负责其他类型的活动。
size：慈善规模，有big,small,mid三种。
program_exp：以美元计的计划费用，即该机构在其提供的计划和服务上花费的金额。
fund_exp：以美元计的资金费用，即该机构筹集资金的金额。
admin_exp：行政费用（美元），即该机构管理费用、员工、会议费用。
#### 三、测试集
测试集中共有1000个样本，每个样本有22个特征，没有训练集中的score这一项特征。
#### 四、要求说明
1.训练集与测试集中均有部分特征的值缺失，需要对缺失特征值做相应的处理。
2.有许多特征是文本特征，如需要利用这些特征，则需要进行预处理。
#### 五、提交注意事项
1.提交作业链接是：学习通
2.提交预测结果的格式为：学号+姓名.csv（如20170000.csv）。文件内容如表1-1所示。预测结果文件非常重要，请一定要保证提交的工程里面有这个文件。

|ein	| score|
|-----------|------|
|93-0642086 | 91.94|
|31-1770828 | 85.59|
**表1-1**

#### 六、评价标准
将用结果的均方根误差(RMSE)来评价模型的好坏。
$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$
其中:
$$ \text{RMSE}  是均方根误差（Root Mean Squared Error）的缩写。$$
$$ n 表示样本的数量。$$ 
$$ y_i  是第i个样本的真实值。$$
$$\hat{y}_i 是第i个样本的预测值。$$
$$(y_i - \hat{y}_i)^2  是第i个样本的预测误差的平方。$$
RMSE 用于衡量回归模型预测值与真实值之间的偏差，值越小表示模型的预测性能越好。公式通过计算所有预测误差的平方和，取平均值，再开平方根，得出最终的误差值。

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']



# 文件路径
train_csv_path = "../data/train.csv"
test_csv_path = "../data/test.csv"
submission_csv_path = "../data/submission.csv"

# 分析编码格式
train_encoding = detect_encoding(train_csv_path)
test_encoding = detect_encoding(test_csv_path)
print(f"Train CSV 编码: {train_encoding}")
print(f"Test CSV 编码: {test_encoding}")


# 读取数据
train_df = pd.read_csv(train_csv_path, encoding=train_encoding)
test_df = pd.read_csv(test_csv_path, encoding=test_encoding)

# 处理缺失值
numeric_features = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train_df.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# 分离特征和标签
X = train_df.drop(['score'], axis=1)
y = train_df['score']

# 预处理和模型pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor())])

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
model.fit(X_train, y_train)

# 预测验证集
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'Validation RMSE: {rmse}')

# 预测测试集
test_predictions = model.predict(test_df)

# 保存预测结果
output = pd.DataFrame({'id': test_df.index, 'score': test_predictions})
output.to_csv(submission_csv_path, index=False)


Train CSV 编码: utf-8
Test CSV 编码: UTF-8-SIG


ValueError: A given column is not a column of the dataframe